In [27]:
from typing import Dict, List, Tuple, Union, Any, Optional

import os
import sys
import time
print("Python version", sys.version)

# Ignore pandas warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pathlib import Path

import pandas as pd
import numpy as np
import tensorflow as tf

import ray
from ray import train, tune

from src.datasets import daocensus

%load_ext autoreload
%autoreload 2

tf.get_logger().setLevel('WARNING')

tf.config.list_physical_devices("GPU")
sys_details = tf.sysconfig.get_build_info()
cuda = sys_details.get("cuda_version", -1)
cudnn = sys_details.get("cudnn_version", -1)
print(cuda, cudnn)

Python version 3.9.18 (main, Oct 24 2023, 09:18:18) 
[GCC 11.4.0]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
11.8 8


In [28]:
# Others config
SEED: int = 57
RAY_RESULTS_PATH: Path = Path('~/ray_results').expanduser()

# Dataset splits config
N_SPLITS: int = 10
SKIP_SPLIT: int = 0

# Training config
MAX_EPOCHS: int = 200
EPOCHS_PER_ITER: int = 5

# Eval config
TOP_K: List[int] = [5, 10]
METRICS: List[str] = ["recall", "ndcg", "precision", "map"]

# Load data

In [29]:
dfptext = pd.read_csv('./snapshot_proposals.csv')[['proposal_id', 'title', 'description', 'start', 'end']]
dfv, dfp = daocensus.get("./data/daos-census", 'Decentraland', 'snapshot')
dfv['voter'] = dfv['voter'].astype('str')
dfp = dfp.merge(dfptext, how='left', left_on='platform_proposal', right_on='proposal_id')
dfp[['start', 'end']] = dfp[['start', 'end']].astype('datetime64')
print(dfv.info())
print(dfp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116560 entries, 0 to 116559
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   platform       116560 non-null  object        
 1   name           116560 non-null  object        
 2   id             116560 non-null  object        
 3   proposal       116560 non-null  category      
 4   deployment     116560 non-null  object        
 5   platform_vote  116560 non-null  object        
 6   voter          116560 non-null  object        
 7   date           116560 non-null  datetime64[ns]
 8   choice         116560 non-null  object        
 9   weight         116560 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1), object(7)
memory usage: 8.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1942 entries, 0 to 1941
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------ 

## Transform data

In [30]:
def to_microsoft(dfv):
    df = dfv[['voter', 'proposal', 'date']].rename(columns={
        'voter': 'userID',
        'proposal': 'itemID',
        'date': 'timestamp',
    })
    df['itemID'] = df['itemID'].astype('str')
    df['rating'] = 1
    return df

df = to_microsoft(dfv)
df.head()

,userID,itemID,timestamp,rating
0,0xe7af1c70f8f089c4c3bd71999692c6c5a15d9e2a,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 12:28:01,1
1,0xc54a6c3778016b06cbd126ccc3b5bc06c5f666fb,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 02:16:23,1
2,0xd82d005e8f8d5385db40ba23884a5c967bb1e8af,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 00:38:22,1
3,0xf4c64db66ffb301985f5ecd85c8f3f9c02f2659d,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:47:08,1
4,0xd5e9ef1cedad0d135d543d286a2c190b16cbb89e,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:32:15,1


# Split data

Each proposal remains open for a few days, our environment is different of a movies recommender system. For this reason, we will use a TimeSeriesSplit instead of a K-Fold to cross-validate the model.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png)

El TimeSeriesSplit de scikit-learn no nos vale porque el número de elementos en cada split es el mismo, pero el tamaño del intervalo, no. Como queremos simular un comportamiento realista, haremos el split dividiendo por intervalos de igual longitud.

In [31]:
from recommenders.evaluation.python_evaluation import metrics as metrics_dict

In [32]:
from src.model_selection import timeIntervalSplit, timeIntervalSplitCurrent

max_train_prev = df['timestamp'].min().date()
folds = list(timeIntervalSplitCurrent(df, N_SPLITS, dfp, skip=SKIP_SPLIT, remove_not_in_train_col='userID', return_open=True))
for i, (dftrain, dftest, t, open_proposals) in enumerate(folds):
    min_train = dftrain['timestamp'].min().date()
    max_train = dftrain['timestamp'].max().date()
    min_test  = dftest['timestamp'].min().date()
    max_test  = dftest['timestamp'].max().date()
    train_diff = (max_train-max_train_prev).days
    test_diff = (max_test-min_test).days

    train_users = len(set(dftrain['userID']))
    test_users = len(set(dftest['userID']))
    
    print(f"Split {i}, train from: {max_train_prev} to {max_train}, test from: {min_test} to {max_test}")
    print(f"  len(train): {len(dftrain)}, len(test): {len(dftest)}")
    print(f"  users(train): {train_users}, users(test): {test_users}")

    print()
    dftest['prediction'] = 1
    for m in METRICS:
        f = metrics_dict[f'{m}_at_k']
        print(f"  highest possible {m}@{TOP_K[0]}:\t{f(dftest, dftest, k=TOP_K[0], relevancy_method='top_k'):.4f}")

    print("-"*30)

    max_train_prev = max_train

Split 0, train from: 2021-05-24 to 2021-08-04, test from: 2021-08-04 to 2021-08-10
  len(train): 3453, len(test): 48
  users(train): 581, users(test): 27

  highest possible recall@5:	0.9894
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.3407
  highest possible map@5:	0.9894
------------------------------
Split 1, train from: 2021-08-04 to 2021-10-14, test from: 2021-10-15 to 2021-10-19
  len(train): 6397, len(test): 132
  users(train): 930, users(test): 121

  highest possible recall@5:	0.9976
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.2149
  highest possible map@5:	0.9976
------------------------------
Split 2, train from: 2021-10-14 to 2021-12-25, test from: 2021-12-25 to 2021-12-31
  len(train): 15187, len(test): 99
  users(train): 2388, users(test): 47

  highest possible recall@5:	0.9721
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.3660
  highest possible map@5:	0.9721
------------------------------
Split 3, tra

# Defining training

In [33]:
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.utils.python_utils import get_top_k_scored_items

class LightGCNCustom(LightGCN):
    # Copied from LightGCN.fit but RETURNING the data and deleting unnecessary things
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.epochs_done = 0
    
    def fit_epoch(self):
        """Fit the model on self.data.train.
        """
        loss, mf_loss, emb_loss = 0.0, 0.0, 0.0
        n_batch = self.data.train.shape[0] // self.batch_size + 1
        for idx in range(n_batch):
            users, pos_items, neg_items = self.data.train_loader(self.batch_size)
            _, batch_loss, batch_mf_loss, batch_emb_loss = self.sess.run(
                [self.opt, self.loss, self.mf_loss, self.emb_loss],
                feed_dict={
                    self.users: users,
                    self.pos_items: pos_items,
                    self.neg_items: neg_items,
                },
            )
            loss += batch_loss / n_batch
            mf_loss += batch_mf_loss / n_batch
            emb_loss += batch_emb_loss / n_batch

        if np.isnan(loss):
            print("ERROR: loss is nan.")
            sys.exit()

        self.epochs_done += 1

        return loss, mf_loss, emb_loss

    def recommend_k_items(
        self, test, top_k=10, sort_top_k=True, remove_seen=True, use_id=False, recommend_from=None,
    ):
        """
        Copy-pasted from LightGCN but adding the `recommend_from` argument
        """
        data = self.data
        if not use_id:
            user_ids = np.array([data.user2id[x] for x in test[data.col_user].unique()])
        else:
            user_ids = np.array(test[data.col_user].unique())

        test_scores = self.score(user_ids, remove_seen=remove_seen)

        ### START NEW BEHAVIOUR
        if recommend_from is not None:
            from_idx = np.array([data.item2id[x] for x in set(recommend_from)])
            msk = np.ones(test_scores.shape[1], bool)
            msk[from_idx] = False

            # Set the score of that proposal to zero for every user
            test_scores[:, msk] = -np.inf
        ### END NEW BEHAVIOUR

        top_items, top_scores = get_top_k_scored_items(
            scores=test_scores, top_k=top_k, sort_top_k=sort_top_k
        )

        df = pd.DataFrame(
            {
                data.col_user: np.repeat(
                    test[data.col_user].drop_duplicates().values, top_items.shape[1]
                ),
                data.col_item: top_items.flatten()
                if use_id
                else [data.id2item[item] for item in top_items.flatten()],
                data.col_prediction: top_scores.flatten(),
            }
        )

        return df.replace(-np.inf, np.nan).dropna()

## Small test of `LightGCNCustom`

In [34]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

In [35]:
hparams = prepare_hparams(
    model_type='lightgcn',
    n_layers=3,
    batch_size=512,
    embed_size=64,
    epochs=2,
    learning_rate=0.001,
    decay=0.001,
    metrics=["recall", "ndcg", "precision", "map"],
    eval_epoch=2,
    top_k=TOP_K[0],
    save_model=False,
    MODEL_DIR='./data/model/lightgcn/',
)
dataloader = ImplicitCF(train=folds[-1][0], test=folds[-1][1], seed=SEED)
print("items:", dataloader.n_items, "user:", dataloader.n_users)
model = LightGCNCustom(data=dataloader, hparams=hparams)

items: 1836 user: 6857
Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [36]:
model.fit()

Epoch 1 (train)0.7s: train loss = 0.60012 = (mf)0.59938 + (embed)0.00075
Epoch 2 (train)0.5s + (eval)0.1s: train loss = 0.35302 = (mf)0.34912 + (embed)0.00389, recall = 0.00400, ndcg = 0.00394, precision = 0.00320, map = 0.00200


In [37]:
model.run_eval()

[0.004, 0.00394083821985168, 0.0032, 0.002]

In [38]:
model.recommend_k_items(dataloader.test, top_k=3, use_id=True, remove_seen=True, recommend_from={'b86aa059-3d31-5d41-a472-70962816f779', '56b4d333-4138-5aa3-9890-3502b9478079', 'd083109e-4819-54b9-a01c-67bd5a770f65' })

,userID,itemID,prediction
0,293,1175,6.377600
1,293,1835,1.713224
2,293,0,-4.863316
3,466,1835,1.273528
4,466,0,-1.401709
...,...,...,...
369,245,1835,1.812435
370,245,0,-5.404130
372,578,1175,3.914507
373,578,1835,0.961036


# Defining trainer

In [39]:
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

class TrainLightGCN(tune.Trainable):
    def setup(
        self,
        config: Dict[str, Any],
        folds: List[Tuple[pd.DataFrame, pd.DataFrame]],
    ):
        self.config = config
        config['batch_size'] = 2**config['batch_size']
        # self.fold = config['__trial_index__']
        self.fold = config['fold']

        self.hparams = prepare_hparams(
            model_type='lightgcn',
            n_layers=config['conv_layers'],
            batch_size=config['batch_size'],
            embed_size=config['embedding_dim'],
            epochs=EPOCHS_PER_ITER,
            learning_rate=config['learning_rate'],
            decay=config['l2'],
            metrics=METRICS,
            eval_epoch=-1,
            top_k=TOP_K[0],
            save_model=False,
            MODEL_DIR='./data/model/lightgcn/',
        )

        train, test, self.t, self.open_proposals = folds[self.fold]
        self.dataloader = ImplicitCF(train=train, test=test, seed=SEED)
        self.model = LightGCNCustom(self.hparams, self.dataloader, seed=SEED)
        self.total_train = 0
        self.total_eval = 0

    @property
    def iteration(self):
        return self.model.epochs_done

    @property
    def training_iteration(self):
        return self.model.epochs_done

    def step(self):
        """
        As a rule of thumb, the execution time of step should be large enough to avoid overheads 
        (i.e. more than a few seconds), but short enough to report progress periodically 
        (i.e. at most a few minutes).
        """
        assert EPOCHS_PER_ITER > 0

        train_start = time.time()
        for _ in range(EPOCHS_PER_ITER):
            ret = self.model.fit_epoch()
        eval_start = train_end = time.time()

        eval_dict = {'model_'+k:v for k,v in zip(self.model.metrics, self.model.run_eval())}
        for k in TOP_K:
            recs = self.model.recommend_k_items(
                self.dataloader.test, 
                top_k=k,
                use_id=True, 
                remove_seen=True, 
                recommend_from=self.open_proposals,
            )
            
            eval_dict[f'precision@{k}'] = precision_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'ndcg@{k}'] = ndcg_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'recall@{k}'] = recall_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'map@{k}'] = map_at_k(self.dataloader.test, recs, k=k)

        eval_end = time.time()

        self.total_train += train_end - train_start
        self.total_eval += eval_end - eval_start
        
        return {
            'iteration': self.iteration,
            'loss': ret[0],
            'mf_loss': ret[1],
            'emb_loss': ret[2],
            **eval_dict,
            'time_train': train_end-train_start,
            'time_test': eval_end-eval_start,
            'time_total_train': self.total_train,
            'time_total_test': self.total_eval,
        }

    def save_checkpoint(self, checkpoint_dir):
        checkpoint_path = os.path.join(checkpoint_dir, "model")
        self.model.saver.save(
            sess=self.model.sess,
            save_path=checkpoint_path,
        )
        return checkpoint_dir

    def load_checkpoint(self, checkpoint_path):
        self.model.load(checkpoint_path)

## Small test of `TrainLightGCN`

In [40]:
model = LightGCNCustom(
    hparams,
    dataloader,
    seed=SEED,
)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [41]:
model.fit()

Epoch 1 (train)0.6s: train loss = 0.60165 = (mf)0.60090 + (embed)0.00075
Epoch 2 (train)0.4s + (eval)0.1s: train loss = 0.35004 = (mf)0.34610 + (embed)0.00394, recall = 0.00400, ndcg = 0.00394, precision = 0.00320, map = 0.00200


# Big experiment

In [42]:
os.uname().nodename

'lamarck'

In [43]:
RAY_RESULTS_PATH

PosixPath('/home/daviddavo/ray_results')

In [44]:
import ray
from ray.tune.search.hyperopt import HyperOptSearch

paths = list(RAY_RESULTS_PATH.glob('TrainLightGCN_*'))
last_experiment = max(paths, key=lambda x: x.stat().st_ctime) if paths else None
last_experiment = None

### SET TRAINING RESOURCES
if os.uname().nodename == 'lamarck':
    # assert torch.cuda.is_available()
    
    NUM_SAMPLES = 500
    # Every run takes approx half a gig of vram (no optimizations)
    # The RTX 4090 has 24GB so we can run the model about 48 times
    resources_per_trial={
        'cpu': 1,
        # GPU has 25GiB, and each run might take up to 2GiB (torch version was lighter)
        # so each run might take up to 1/12th of the GPU
        # I use 1/16th so I don't take all the resources in the machine
        'gpu': 1/16,
    }
else:
    NUM_SAMPLES = 1
    resources_per_trial={
        'cpu': 1,
        # It takes about 1.5 GiB with full training data, but I put a bit more because
        # this notebook also takes a bit of memory
        'memory': 2e9,
    }

### RESTORE EXPERIMENT OR CREATE A NEW ONE
if last_experiment and tune.Tuner.can_restore(last_experiment):
    print(f"Restoring last experiment: {last_experiment}")
    tuner = tune.Tuner.restore(
        str(last_experiment),
        trainable=tune.with_resources(
            tune.with_parameters(TrainLightGCN, folds=folds),
            resources_per_trial,
        ),
        restart_errored=True
    )
else:
    print("No experiment found, creating new tuner")
    # search_alg = HyperOptSearch()
    # search_alg = tune.search.Repeater(None, N_SPLITS-SKIP_SPLIT)
    search_alg = None
    
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(TrainLightGCN, folds=folds),
            resources_per_trial,
        ),
        run_config=train.RunConfig(
            stop={'training_iteration': MAX_EPOCHS/EPOCHS_PER_ITER, 'time_total_s': 300},
            storage_path=RAY_RESULTS_PATH,
        ),
        param_space=dict(
            # every in this grid will be executed NUM_SAMPLES times
            fold=tune.grid_search(range(N_SPLITS-SKIP_SPLIT)),
            # batch size between 2**4 (32) and 2**10 (1024)
            batch_size=tune.randint(4,10),
            embedding_dim=tune.qlograndint(1, 1000, 5),
            conv_layers=tune.randint(1,6),
            learning_rate=tune.qloguniform(1e-4, 1, 1e-4),
            l2=tune.loguniform(1e-7, 1e-2),
        ),
        tune_config=tune.TuneConfig(
            search_alg=search_alg,
            # num_samples=(N_SPLITS-SKIP_SPLIT)*NUM_SAMPLES,
            num_samples=NUM_SAMPLES,
            metric='loss',
            mode='min',
        )
    )

No experiment found, creating new tuner


In [45]:
tuner.fit()

2023-11-15 14:30:28,520	WARNING tune.py:194 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
(pid=1398125) 2023-11-15 14:30:28.936257: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1398125) 2023-11-15 14:30:28.936293: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1398125) 2023-11-15 14:30:28.936310: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainLightGCN pid=1398125) /home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/

(TrainLightGCN pid=1398125) Already create adjacency matrix.
(TrainLightGCN pid=1398125) Already normalize adjacency matrix.
(TrainLightGCN pid=1398125) Using xavier initialization.


2023-11-15 14:30:33,081	INFO tune.py:1143 -- Total run time: 5.37 seconds (0.80 seconds for the tuning loop).
2023-11-15 14:30:33,082	WARNING tune.py:1158 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/home/daviddavo/ray_results/TrainLightGCN_2023-11-15_14-30-27", trainable=...)
2023-11-15 14:30:33,086	WARNING experiment_analysis.py:205 -- Failed to fetch metrics for 26 trial(s):
- TrainLightGCN_85872_00000: FileNotFoundError('Could not fetch metrics for TrainLightGCN_85872_00000: both result.json and progress.csv were not found at /home/daviddavo/ray_results/TrainLightGCN_2023-11-15_14-30-27/TrainLightGCN_85872_00000_0_batch_size=9,conv_layers=2,embedding_dim=130,fold=0,l2=0.0002,learning_rate=0.0018_2023-11-15_14-30-27')
- TrainLightGCN_85872_00001: FileNotFoundError('Could not fetch metrics for TrainLightGCN_85872_00001: both result.json and progress.csv were not found at /home/daviddavo/ray_results/TrainLightGCN

ResultGrid<[
  Result(
    metrics={},
    path='/home/daviddavo/ray_results/TrainLightGCN_2023-11-15_14-30-27/TrainLightGCN_85872_00000_0_batch_size=9,conv_layers=2,embedding_dim=130,fold=0,l2=0.0002,learning_rate=0.0018_2023-11-15_14-30-27',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={},
    path='/home/daviddavo/ray_results/TrainLightGCN_2023-11-15_14-30-27/TrainLightGCN_85872_00001_1_batch_size=5,conv_layers=5,embedding_dim=235,fold=1,l2=0.0009,learning_rate=0.0064_2023-11-15_14-30-27',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={},
    path='/home/daviddavo/ray_results/TrainLightGCN_2023-11-15_14-30-27/TrainLightGCN_85872_00002_2_batch_size=5,conv_layers=3,embedding_dim=15,fold=2,l2=0.0000,learning_rate=0.1795_2023-11-15_14-30-27',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={},
    path='/home/daviddavo/ray_results/TrainLightGCN_2023-11-15_14-30-27/TrainLightGCN_85872_00003_3_batch_size=6,conv_laye